In [2]:
import cv2
import mediapipe as mp
import numpy as np

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.7, min_tracking_confidence=0.7)

cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    result = hands.process(frame_rgb)

    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            h, w, _ = frame.shape

            for fingertip_id, knuckle_id in zip([4, 8, 12, 16, 20], [3, 7, 11, 15, 19]):
                x_tip = int(hand_landmarks.landmark[fingertip_id].x * w)
                y_tip = int(hand_landmarks.landmark[fingertip_id].y * h)
                x_knuckle = int(hand_landmarks.landmark[knuckle_id].x * w)
                y_knuckle = int(hand_landmarks.landmark[knuckle_id].y * h)

                dx = x_tip - x_knuckle
                dy = y_tip - y_knuckle

                nail_x = x_tip + int(dx * 0.3)
                nail_y = y_tip + int(dy * 0.3)

                nail_region = frame[max(0, nail_y-15):min(h, nail_y+15), max(0, nail_x-15):min(w, nail_x+15)]

                cv2.rectangle(frame, (nail_x-15, nail_y-15), (nail_x+15, nail_y+15), (0, 255, 0), 2)

                if nail_region.size > 0:
                    cv2.imshow(f"Nail {fingertip_id}", nail_region)

    cv2.imshow("Hand and Nails Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()